In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest


In [15]:
# Generate synthetic data for keystroke dynamics (Replace with real data)
def generate_synthetic_data(num_samples=1000):
    np.random.seed(42)
    keystroke_latency = np.random.normal(100, 20, num_samples)  # in ms
    key_hold_time = np.random.normal(50, 10, num_samples)  # in ms
    flight_time = np.random.normal(80, 15, num_samples)  # in ms
    typing_speed = np.random.normal(200, 30, num_samples)  # characters per minute
    return np.column_stack([keystroke_latency, key_hold_time, flight_time, typing_speed])

# Load and preprocess data
data = generate_synthetic_data()
data

array([[109.93428306,  63.99355437,  69.87232588, 142.76577326],
       [ 97.23471398,  59.24633683,  77.83221994, 174.18844968],
       [112.95377076,  50.5963037 ,  68.11370119, 187.591834  ],
       ...,
       [112.81685723,  41.18125349,  61.37359137, 190.39106586],
       [ 88.5764202 ,  48.36933036,  85.01264626, 249.30134467],
       [111.45165563,  42.55097355,  77.67111427, 210.81943674]],
      shape=(1000, 4))

In [16]:

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_scaled

array([[ 0.48775857,  1.33257619, -0.6928164 , -1.84010685],
       [-0.1610219 ,  0.85640475, -0.15295893, -0.81984254],
       [ 0.64201457, -0.01124009, -0.81209018, -0.38464745],
       ...,
       [ 0.63502009, -0.95562025, -1.2692192 , -0.29375906],
       [-0.60334651, -0.23461749,  0.33403332,  1.61900145],
       [ 0.5652763 , -0.81822996, -0.16388547,  0.36953064]],
      shape=(1000, 4))

In [17]:

# Train Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(data_scaled)
iso_forest

IsolationForest(contamination=0.05, random_state=42)

In [18]:

def detect_user_legitimacy(new_sample):
    new_sample_scaled = scaler.transform([new_sample])
    prediction = iso_forest.predict(new_sample_scaled)
    return "Legitimate" if prediction[0] == 1 else "Anomalous"


In [19]:
# Example real-time analysis
new_user_data = [110, 45, 75, 190]  # Example new user's keystroke behavior
result = detect_user_legitimacy(new_user_data)
print("User is:", result)

User is: Legitimate


In [22]:
# Example real-time analysis
new_user_data = [50, 30, 25, 20]  # Example new user's keystroke behavior
result = detect_user_legitimacy(new_user_data)
print("User is:", result)

User is: Anomalous


In [ ]:
import redis
import json
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)
user_id="123"
key = f"user:{user_id}:behavior"
redis_client.rpush(f"user:{user_id}:behavior", json.dumps({"user_id": "123", "keystroke_latency": 0, "key_hold_time": 38.29999999701977, "flight_time": 813223.8999999985, "typing_speed": 12}))

data_list = redis_client.lrange(key, 0, -1)
redis_client.delete(key)  # Clear Redis queue after processing

if not data_list:
    print( "No behavior data found for user" )

behavior_data = [json.loads(data) for data in data_list]
features = np.array([[d["keystroke_latency"], d["key_hold_time"], d["flight_time"], d["typing_speed"]] for d in behavior_data])
features
    # Normalize and predict

No behavior data found for user


array([], dtype=float64)